In [7]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import time
import joblib
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import os
from stable_baselines3 import PPO


In [68]:
# Initialize MT5
if not mt5.initialize():
    raise RuntimeError("❌ MT5 initialization failed")

MT5_TIMEFRAME = mt5.TIMEFRAME_M5
symbols = ["EURUSD", "GBPUSD", "XAUUSD"]

print("MT5 Connected:", mt5.initialize())


MT5 Connected: True


In [69]:
model_path = "models/multiasset/ppo_multiasset.zip"
model = PPO.load(model_path)
print("Model loaded.")


Model loaded.


In [70]:
import os
import pandas as pd

scalers = {}

folder = "data/multiasset"
for f in os.listdir(folder):
    if f.endswith("_scaler.csv"):
        safe = f.replace("_scaler.csv", "")
        scalers[safe] = pd.read_csv(os.path.join(folder, f), index_col=0).to_dict()['mean']
        # Actually read BOTH mean and std correctly
        df = pd.read_csv(os.path.join(folder, f), index_col=0)
        scalers[safe] = {
            "mean": df["mean"],
            "std": df["std"]
        }


In [80]:
safe_names = list(scalers.keys())
print("Safe symbols:", safe_names)


Safe symbols: ['EURUSD', 'Jump_100_Index', 'Jump_10_Index', 'Jump_25_Index', 'Jump_50_Index', 'Jump_75_Index', 'Volatility_100_1s_Index', 'Volatility_100_Index', 'Volatility_10_1s_Index', 'Volatility_10_Index', 'Volatility_25_1s_Index', 'Volatility_25_Index', 'Volatility_50_1s_Index', 'Volatility_50_Index', 'Volatility_75_1s_Index', 'Volatility_75_Index']


In [72]:
embeddings_path = "models/multiasset/asset_embeddings.npy"
embeddings = np.load(embeddings_path, allow_pickle=True)
#embeddings = np.load(embeddings_path, allow_pickle=True).item()
print("Embeddings loaded.")


Embeddings loaded.


In [ ]:
#EMBED_FILE = os.path.join(MODEL_DIR, "asset_embeddings.npy")

In [67]:
MODEL_DIR = os.path.join("models", "multiasset")
model_path = "models/multiasset/ppo_multiasset.zip"
scalers_path = os.path.join("data/multiasset")
embeddings_path = os.path.join(MODEL_DIR, "asset_embeddings.npy")
#safe_names_path = "safe_names.pkl"


model = PPO.load(model_path)
#scalers = scalers_path
#embeddings = embeddings_path
#safe_names = joblib.load(safe_names_path)

print("✅ Model + preprocessing loaded.")


✅ Model + preprocessing loaded.


In [73]:
def fetch_and_build_obs(symbol, window, scalers, embeddings, safe_names):
    safe = symbol.replace(" ", "_").replace("/", "_")

    if safe not in scalers:
        print(f"❌ Missing scaler for {safe}")
        return None, None, None

    if safe not in embeddings:
        print(f"❌ Missing embedding for {safe}")
        return None, None, None

    scaler = scalers[safe]
    embed = embeddings[safe]

    bars = mt5.copy_rates_from_pos(symbol, MT5_TIMEFRAME, 0, window+20)
    if bars is None or len(bars) < window+5:
        return None, None, None

    df = pd.DataFrame(bars)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df = df.set_index('time')
    df = df[['open','high','low','close','tick_volume']]
    df.rename(columns={'tick_volume':'volume'}, inplace=True)

    pct = df.pct_change().dropna()
    pct = pct.tail(window)

    pct_norm = (pct - scaler["mean"]) / scaler["std"]

    last_price = df["close"].iloc[-1]
    vol_est = pct["close"].std()

    bal = np.full((window, 1), 1.0, dtype=np.float32)
    asset_id = safe_names.index(safe) / len(safe_names)
    asset_id_column = np.full((window, 1), asset_id, dtype=np.float32)

    emb = np.tile(embed, (window, 1)).astype(np.float32)

    obs = np.column_stack([
        pct_norm.values.astype(np.float32),
        emb,
        bal,
        asset_id_column
    ]).astype(np.float32)

    return obs, float(vol_est), float(last_price)


In [74]:
def pip_value(symbol):
    if "JPY" in symbol:
        return 0.01
    return 0.0001


In [75]:
def compute_sl_tp(symbol, price, action, sl_pips=20, tp_pips=40):
    pip = pip_value(symbol)

    if action == 1:  # BUY
        sl = price - sl_pips * pip
        tp = price + tp_pips * pip
    elif action == 2:  # SELL
        sl = price + sl_pips * pip
        tp = price - tp_pips * pip
    else:
        sl, tp = None, None

    return sl, tp


In [76]:
def apply_trailing_stop(symbol, position, trail_pips=15):
    pip = pip_value(symbol)
    current_price = mt5.symbol_info_tick(symbol).bid if position.type == 0 else mt5.symbol_info_tick(symbol).ask

    if position.type == 0:  # BUY
        new_sl = current_price - trail_pips * pip
        if new_sl > position.sl:
            return new_sl

    if position.type == 1:  # SELL
        new_sl = current_price + trail_pips * pip
        if new_sl < position.sl:
            return new_sl

    return None


In [77]:
def get_symbol_positions(symbol):
    return [p for p in mt5.positions_get(symbol=symbol) or []]

def close_position(position):
    symbol = position.symbol
    lot = position.volume

    if position.type == 0:
        price = mt5.symbol_info_tick(symbol).bid
        order_type = mt5.ORDER_TYPE_SELL
    else:
        price = mt5.symbol_info_tick(symbol).ask
        order_type = mt5.ORDER_TYPE_BUY

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": order_type,
        "price": price,
        "magic": 12345,
        "comment": "Auto-close",
        "deviation": 10,
    }

    mt5.order_send(request)


In [78]:
def place_trade(symbol, action, lot=0.01):
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        return

    price = tick.ask if action == 1 else tick.bid

    sl, tp = compute_sl_tp(symbol, price, action)

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY if action == 1 else mt5.ORDER_TYPE_SELL,
        "price": price,
        "sl": sl,
        "tp": tp,
        "magic": 12345,
        "comment": "RL-AutoTrade",
        "deviation": 10,
    }

    return mt5.order_send(request)


In [79]:
WINDOW = 32
actions_dict = {0: "HOLD", 1: "BUY", 2: "SELL"}

while True:
    for symbol in symbols:
        obs, vol, last_price = fetch_and_build_obs(symbol, WINDOW, scalers, embeddings, safe_names)

        if obs is None:
            continue

        action, _ = model.predict(obs, deterministic=True)
        action = int(action)

        positions = get_symbol_positions(symbol)

        # 1️⃣ Auto-close on reverse signal
        for p in positions:
            if (p.type == 0 and action == 2) or (p.type == 1 and action == 1):
                close_position(p)

        # 2️⃣ Enforce max 2 positions per symbol
        if len(positions) >= 2:
            continue

        # 3️⃣ Open new trades
        if action in [1, 2]:
            place_trade(symbol, action)

        # 4️⃣ Trailing stop
        for p in get_symbol_positions(symbol):
            new_sl = apply_trailing_stop(symbol, p)
            if new_sl:
                mt5.order_send({
                    "action": mt5.TRADE_ACTION_SLTP,
                    "symbol": symbol,
                    "position": p.ticket,
                    "sl": new_sl,
                    "tp": p.tp,
                })

    time.sleep(5)


❌ Missing embedding for EURUSD
❌ Missing scaler for GBPUSD
❌ Missing scaler for XAUUSD
❌ Missing embedding for EURUSD
❌ Missing scaler for GBPUSD
❌ Missing scaler for XAUUSD
❌ Missing embedding for EURUSD
❌ Missing scaler for GBPUSD
❌ Missing scaler for XAUUSD


KeyboardInterrupt: 

In [ ]:
true_labels = []
pred_labels = []

for symbol in symbols:
    bars = mt5.copy_rates_from_pos(symbol, MT5_TIMEFRAME, 0, 500)
    df = pd.DataFrame(bars)
    df["return"] = df["close"].pct_change().shift(-1)
    df["true_trend"] = np.where(df["return"] > 0, 1, 2)  # 1=BUY, 2=SELL

    for i in range(100, 400):
        window_df = df.iloc[i-32:i]
        scaler = scalers[symbol.replace("/","_")]
        embed = embeddings[symbol.replace("/","_")]

        obs, _, _ = fetch_and_build_obs(symbol, WINDOW, scalers, embeddings, safe_names)
        action, _ = model.predict(obs)

        pred_labels.append(action)
        true_labels.append(df["true_trend"].iloc[i])


In [ ]:
print("Accuracy:", accuracy_score(true_labels, pred_labels))
print("Precision:", precision_score(true_labels, pred_labels, average='macro'))
print("Recall:", recall_score(true_labels, pred_labels, average='macro'))
print("F1:", f1_score(true_labels, pred_labels, average='macro'))
print("Confusion Matrix:")
print(confusion_matrix(true_labels, pred_labels))
